In [8]:
import pandas as pd
import streamlit as st
from fpdf import FPDF

from datetime import datetime
import io
import os
import base64
import openpyxl


def lectura_csv(path):
    # Lectura del archivo csv con los datos de cada paciente.
    df = pd.read_csv(path, sep=';')
    # Los datos se pasan a un diccionario
    datos_dict = {df["Sample/Assay"][i]: {df.columns[j]: df[df.columns[j]][i] for j in range(1, len(df.columns))} for i in range(len(df))}
    # Tabla con las variantes y sus mutaciones asociadas
    tabla = {"CYP2D6": {"3": ["_"], "4": ["T"], "6": [ "_"], "7": ["G"],
                    "8": ["A"], "9": ["_"], "10*4": ["A"], "10": ["G"],
                    "12": ["T"], "14": ["T"], "15": ["_"], "17": ["A"],
                    "19": ["_"], "29": ["A"], "41": ["T"], "56B": ["A"],
                    "59": ["T"]},
         "UGT1A1": {"80": ["T"]},
         "DPYD": {"2A": [ "G","T" ], "13": ["C", "T"], "HapB3": ["T"], "D949V": ["A"]}}

    dict_pacientes = {}


    for i in datos_dict:
        dict_pacientes[i] = {}
        for j in datos_dict[i]:
            # Separacion de nombre de gen y variantes
            if "*" in j:
                separador = "*"
            elif "_" in j:
                separador = "_"
            snp = j.split(separador, 1)


            if snp[0] not in dict_pacientes[i]:
                dict_pacientes[i][snp[0]] = []

            # Manejo de Indeterminados
            if datos_dict[i][j] == "UND":
                dict_pacientes[i][snp[0]].append(("*1", "*1"))
                continue
            # Determinar genotipo 
            variante = []
            if datos_dict[i][j].split("/")[0] in tabla[snp[0]][snp[1]]:
                variante.append(f"{separador}{snp[1]}")
            else:
                variante.append(f"*1")

            if datos_dict[i][j].split("/")[1] in tabla[snp[0]][snp[1]]:
                variante.append(f"{separador}{snp[1]}")
            else:
                variante.append(f"*1")

            dict_pacientes[i][snp[0]].append(tuple(variante))

    # Eliminar duplicados
    for i in dict_pacientes:
        for j in dict_pacientes[i]:
            dict_pacientes[i][j] = list(set(dict_pacientes[i][j]))

    return dict_pacientes

def determinar_genotipo_definitivo(datos_pacientes):
    """
    Determina el genotipo definitivo para cada gen de cada paciente
    """
    resultados = {}
    
    for paciente, genes in datos_pacientes.items():
        resultados[paciente] = {}
        
        for gen, alelos in genes.items():
            # Recoger todos los alelos únicos, excluyendo *1 cuando hay otros
            alelos_maternos_unicos = set()
            alelos_paternos_unicos = set()

            for alelo_materno, alelo_paterno in alelos:
                # Si ambos son *1, es el caso base (sin mutaciones)
                if alelo_materno == '*1' and alelo_paterno == '*1':
                    continue
                
                # Añadir alelos no silvestres
                
                if alelo_materno != '*1':
                    alelos_maternos_unicos.add(alelo_materno)
                if alelo_paterno != '*1':
                    alelos_paternos_unicos.add(alelo_paterno)

            if gen == "CYP2D6":
                # Manejo de Variante *10*4 Materno
                if "*10*4" in alelos_maternos_unicos and "*10" in alelos_maternos_unicos:
                    alelos_maternos_unicos.remove("*10*4")
                    if "*4" in alelos_maternos_unicos:
                        alelos_maternos_unicos.remove("*10")
                # Manejo de Variante *10*4 Paterno 
                if "*10*4" in alelos_paternos_unicos and "*10" in alelos_paternos_unicos:
                    alelos_paternos_unicos.remove("*10*4")
                    if "*4" in alelos_paternos_unicos:
                        alelos_paternos_unicos.remove("*10")
                #Manejo si estan en alelos distintos
                if "*10*4" in alelos_paternos_unicos and "*10" in alelos_maternos_unicos:
                    alelos_paternos_unicos.remove("*10*4")
                    if "*4" in alelos_maternos_unicos:
                        alelos_maternos_unicos.remove("*10")
                if "*10*4" in alelos_maternos_unicos and "*10" in alelos_paternos_unicos:
                    alelos_maternos_unicos.remove("*10*4")
                    if "*4" in alelos_paternos_unicos:
                        alelos_paternos_unicos.remove("*10")


            # Manejo de variante *80 como *28
            if gen == "UGT1A1":
                if alelos_maternos_unicos:
                    alelos_maternos_unicos = list(alelos_maternos_unicos)
                    alelos_maternos_unicos[0] = "*28"
                    alelos_maternos_unicos = set(alelos_maternos_unicos)
                if alelos_paternos_unicos:
                    alelos_paternos_unicos = list(alelos_paternos_unicos)
                    alelos_paternos_unicos[0] = "*28"
                    alelos_paternos_unicos = set(alelos_paternos_unicos)

            # Si no hay mutaciones, el genotipo es *1/*1
            if not alelos_maternos_unicos and not alelos_paternos_unicos:
                resultados[paciente][gen] = ('*1', '*1')
            # Si hay una mutación, es heterocigoto *1/mutación
            elif alelos_maternos_unicos and not alelos_paternos_unicos:
                mutacion = list(alelos_maternos_unicos)[0]
                resultados[paciente][gen] = ('*1', mutacion)
            # Si hay dos mutaciones, es heterocigoto mutación1/mutación2
            elif not alelos_maternos_unicos and alelos_paternos_unicos:
                mutacion = list(alelos_paternos_unicos)[0]
                resultados[paciente][gen] = ('*1', mutacion)
            elif alelos_maternos_unicos and alelos_paternos_unicos:

                mutacion_materna = list(alelos_maternos_unicos)[0]
                mutacion_paterna = list(alelos_paternos_unicos)[0]
                resultados[paciente][gen] = (mutacion_materna, mutacion_paterna)

    return resultados

# Función para formatear el resultado como string
def formatear_genotipos(resultados):
    formateados = {}
    for paciente, genes in resultados.items():
        formateados[paciente] = {}
        for gen, alelos in genes.items():
            formateados[paciente][gen] = f"{alelos[0]}/{alelos[1]}"
    return formateados


# Procesar
dict_pacientes = lectura_csv("Genotype Matrix.csv")
resultados = determinar_genotipo_definitivo(dict_pacientes)
resultados_formateados = formatear_genotipos(resultados)

#print(resultados_formateados)


df = pd.read_excel("CYP2D6_Diplotype_Phenotype_Table.xlsx")
diccionario_CYP2D6 = dict(zip(df.iloc[:, 0], zip(df.iloc[:, 1], df.iloc[:, 2])))
def fenotipo(genotipo):
    Sol = {}
    diccionario = genotipo
    for nombre in diccionario:
        Sol[nombre] = {}
        for gen in diccionario[nombre]:
            alelos = diccionario[nombre][gen].split('/')   
            if gen == 'DPYD' or gen == 'UGT1A1':
                if alelos[0] == "*1" and alelos[1] == "*1":       
                    Sol[nombre][gen] = [f"{alelos[0]}/{alelos[1]}", 2.0, 'Normal Metabolizer']
                elif alelos[0] != "*1" and alelos[1] != "*1":
                    Sol[nombre][gen] = [f"{alelos[0]}/{alelos[1]}", 0.0, 'Poor Metabolizer']
                else:
                    Sol[nombre][gen] = [f"{alelos[0]}/{alelos[1]}", 1.0,'Intermediate Metabolizer']
            else:
                Sol[nombre][gen] = [diccionario[nombre][gen]]
                Sol[nombre][gen].append(diccionario_CYP2D6[diccionario[nombre][gen]][0])
                Sol[nombre][gen].append(diccionario_CYP2D6[diccionario[nombre][gen]][1])
    return Sol




def recomendacionClinica(fenotipo):
    import json # Importa la biblioteca JSON para trabajar con datos JSON.
    import requests # Importa la biblioteca Requests para realizar solicitudes HTTP.
    resultado = fenotipo # Inicializa una lista vacía para almacenar los resultados.
    for paciente in fenotipo:
        for gen in fenotipo[paciente]:
            if gen == "CYP2D6":
                lookupkey= [gen, str(fenotipo[paciente][gen][1])] # Obtiene la clave de búsqueda del fenotipo.
                ID_Farmaco = "RxNorm:10324"
                url='https://api.cpicpgx.org/v1/recommendation?select=drug(name),guideline(name),*&drugid=eq.'+ID_Farmaco+'&lookupkey=cs.{\"'+lookupkey[0]+'":"'+lookupkey[1]+'"}' 
            elif gen == "DPYD":
                lookupkey= [gen, str(fenotipo[paciente][gen][1])] # Obtiene la clave de búsqueda del fenotipo.
                ID_Farmaco = "RxNorm:51499"
                url='https://api.cpicpgx.org/v1/recommendation?select=drug(name),guideline(name),*&drugid=eq.'+ID_Farmaco+'&lookupkey=cs.{\"'+lookupkey[0]+'":"'+lookupkey[1]+'"}' 
                if float(lookupkey[1])==2.0:
                    resultado[paciente][gen].append("Based on genotype, there is no indication to change dose or therapy. Use label-recommended dosage and administration.")
                elif float(lookupkey[1])>=1.0:
                    resultado[paciente][gen].append("Reduce starting dose by 50% followed by titration of dose based on toxicity or therapeutic drug monitoring (if available). Patients with the c.[2846A>T];[2846A>T] genotype may require >50% reduction in starting dose.")
                elif float(lookupkey[1])==0.5:
                    resultado[paciente][gen].append("Avoid use of 5- fluorouracil or 5-fluorouracil prodrug-based regimens. In the event, based on clinical advice, alternative agents are not considered a suitable therapeutic option, 5-fluorouracil should be administered at a strongly reduced dose with early therapeutic drug monitoring.")
                elif float(lookupkey[1])== 0.0:
                    resultado[paciente][gen].append("Avoid use of 5-fluorouracil or 5-fluorouracil prodrug-based regimens.")
            elif gen == "UGT1A1":
                lookupkey= [gen, str(fenotipo[paciente][gen][1])] # Obtiene la clave de búsqueda del fenotipo.
                ID_Farmaco = "RxNorm:51499"
                url='https://api.cpicpgx.org/v1/recommendation?select=drug(name),guideline(name),*&drugid=eq.'+ID_Farmaco+'&lookupkey=cs.{\"'+lookupkey[0]+'":"'+lookupkey[1]+'"}' 
                if fenotipo[paciente][gen][0] == "*1/*1":
                    resultado[paciente][gen].append("The guideline does not provide a recommendation for irinotecan in normal metabolizers.")
                elif fenotipo[paciente][gen][0] == "*1/*28":
                    resultado[paciente][gen].append("NO action is needed for this gene-drug interaction.")
                elif fenotipo[paciente][gen][0] == "*28/*28":
                    resultado[paciente][gen].append("Start with 70% of the normal dose If the patient tolerates this initial dose, the dose can be increased, guided by the neutrophil count.")
            response = requests.get(url) # Realiza una solicitud GET a la API.
            json_obtenido = response.json() # Convierte la respuesta JSON en un objeto Python.
            datos=json_obtenido # Asigna los datos JSON a la variable 'datos'.
            if len(datos) != 0: # Verifica si se encontraron recomendaciones.
                resultado[paciente][gen].append(datos[0]['drugrecommendation'].encode('latin-1','ignore').decode('latin-1')) # Agrega la recomendación del fármaco a la lista, decodificando caracteres especiales.
    return resultado # Devuelve la lista con los resultados.





In [9]:
fenotipo_test = fenotipo(resultados_formateados)
print(fenotipo_test)



{'DPD932': {'CYP2D6': ['*10/*10', '0.5', 'Intermediate Metabolizer'], 'DPYD': ['*1/_HapB3', 1.0, 'Intermediate Metabolizer'], 'UGT1A1': ['*1/*1', 2.0, 'Normal Metabolizer']}, 'DPD933': {'CYP2D6': ['*10/*10', '0.5', 'Intermediate Metabolizer'], 'DPYD': ['*1/*1', 2.0, 'Normal Metabolizer'], 'UGT1A1': ['*1/*28', 1.0, 'Intermediate Metabolizer']}, 'DPD934': {'CYP2D6': ['*1/*4', '1.0', 'Intermediate Metabolizer'], 'DPYD': ['*1/*1', 2.0, 'Normal Metabolizer'], 'UGT1A1': ['*1/*1', 2.0, 'Normal Metabolizer']}, 'DPD935': {'CYP2D6': ['*1/*10', '1.25', 'Normal Metabolizer'], 'DPYD': ['*1/*1', 2.0, 'Normal Metabolizer'], 'UGT1A1': ['*1/*28', 1.0, 'Intermediate Metabolizer']}, 'DPD936': {'CYP2D6': ['*1/*10', '1.25', 'Normal Metabolizer'], 'DPYD': ['*1/*1', 2.0, 'Normal Metabolizer'], 'UGT1A1': ['*1/*28', 1.0, 'Intermediate Metabolizer']}, 'DPD937': {'CYP2D6': ['*1/*10', '1.25', 'Normal Metabolizer'], 'DPYD': ['*1/*1', 2.0, 'Normal Metabolizer'], 'UGT1A1': ['*1/*1', 2.0, 'Normal Metabolizer']}, 'DPD

In [10]:
resultado_final = recomendacionClinica(fenotipo_test)

print(resultado_final)

{'DPD932': {'CYP2D6': ['*10/*10', '0.5', 'Intermediate Metabolizer', 'Consider hormonal therapy such as an aromatase inhibitor for postmenopausal women or aromatase inhibitor along with ovarian function suppression in premenopausal women, given that these approaches are superior to tamoxifen regardless of CYP2D6 genotype (PMID 26211827). If aromatase inhibitor use is contraindicated, consideration should be given to use a higher but FDA approved tamoxifen dose (40 mg/day)(PMID 27226358). Avoid CYP2D6 strong to weak inhibitors.'], 'DPYD': ['*1/_HapB3', 1.0, 'Intermediate Metabolizer', 'Reduce starting dose by 50% followed by titration of dose based on toxicity or therapeutic drug monitoring (if available). Patients with the c.[2846A>T];[2846A>T] genotype may require >50% reduction in starting dose.'], 'UGT1A1': ['*1/*1', 2.0, 'Normal Metabolizer', 'The guideline does not provide a recommendation for irinotecan in normal metabolizers.']}, 'DPD933': {'CYP2D6': ['*10/*10', '0.5', 'Intermed

In [12]:
c = 0
for paciente in resultado_final:
    for gen in resultado_final[paciente]:
        if len(resultado_final[paciente][gen]) != 4:
            print(resultado_final[paciente])
            c+=1

print(c)



0
